In [1]:
# !pip install openai
# !pip install python-dotenv
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables
dotenv_path = find_dotenv()
print(f"Loading .env from: {dotenv_path}")
load_dotenv(dotenv_path)

api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("Missing OPENAI_API_KEY. Put it in your .env file")

# Strip any quotes that might be included
api_key = api_key.strip('"').strip("'")

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

print("✓ Key loaded successfully and client initialized.")

Loading .env from: /Users/mac/Desktop/LLM Chatbot/.env
✓ Key loaded successfully and client initialized.


In [2]:
def get_completion(prompt, model="gpt-5-nano"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1
    )
    return response.choices[0].message.content

def get_message_completion(messages, model="gpt-5-nano", temperature=1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

def chat_with_context(user_message, context):
    """
    Takes user message and context, gets response from LLM,
    and updates context with both user message and assistant response
    """
    # Add user message to context
    context.append({"role": "user", "content": user_message})
    
    # Get response from LLM with full context
    assistant_response = get_message_completion(context)
    
    # Add assistant response to context
    context.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response, context


In [3]:
resp = get_completion("what is a 1+1 in math")
print(resp)


1+1 equals 2.

- In standard base-10 arithmetic, adding 1 and 1 gives 2.
- Addition is just combining quantities.
- In binary (base-2), 1+1 = 10, which is the binary representation of decimal 2.


In [4]:
# !pip install --upgrade gradio
import gradio as gr
import json
from db_functions import get_or_create_coach_state, save_coach_state

print(f"Gradio Version: {gr.__version__}")

# System prompt for the coach
COACH_SYSTEM_PROMPT = """You are a mentor-coach for high-pressure individuals such as founders. For each session, you will receive:
- **COACH_STATE:** a JSON containing the user's long-term goals, plans, blockers, preferences, and commitments. Treat this as authoritative.
- **RECENT_TURNS:** the user's most recent conversation exchanges.
- **The user's latest message.**

Your outputs must adhere to these instructions:

- Use **COACH_STATE** as the reliable source of the user's objectives, blockers, and commitments.
- Use **RECENT_TURNS** for short-term context and recent conversational flow.

For each response, follow this structured format:

1. **Pattern & Stress Signals**: Begin by analyzing the user's latest message for stress indicators, recurring patterns, and potential mindset pitfalls. Reference COACH_STATE and RECENT_TURNS for accuracy.
2. **Recommendations**: Based on the above, provide highly specific, actionable advice that speaks to the user's context, closing the gap between their goals and current situation. Avoid generic or repetitive suggestions.
3. **Next Actions (checklist)**: Provide a short, actionable checklist (2–4 items max) for what the user should do before the next check-in.
4. **Follow-up Questions (if needed)**: Pose 1–3 sharp clarifying questions that help flesh out missing details about goals, blockers, preferences, commitments, or timelines. Only ask if this information is not precise in COACH_STATE or RECENT_TURNS.

**Rules:**
- Do NOT fabricate or infer facts, deadlines, or commitments not present in COACH_STATE or RECENT_TURNS.
- If key information is missing, use targeted follow-up questions to uncover it.
- If the user reports progress, recognize it and tailor new advice precisely, but do not assert that memory has been updated (this process is external).
- Keep language concise, direct, and accountability-focused. Avoid platitudes or soft generalities.

**Output Format**: Human-readable, no JSON or code formatting. Use clear, separate section headers for each part (Pattern & Stress Signals, Recommendations, Next Actions, Follow-up Questions).

**Reminder:** Always ground your responses in the provided COACH_STATE and RECENT_TURNS. Never fabricate information, commitments, or deadlines not found there. Organize your answer using the mandatory output sections.
"""

MEMORY_UPDATER_PROMPT = """
You are a memory updater for a coaching chatbot.
Input:
- OLD_COACH_STATE (JSON)
- DIALOGUE_CHUNK (recent turns from the CURRENT session, in-memory)

Task:
Update OLD_COACH_STATE using ONLY facts explicitly stated in DIALOGUE_CHUNK.
Return ONLY the UPDATED_COACH_STATE as valid JSON. No extra text.

Special requirement:
Update "pattern_analysis" and "last_emotional_state" to reflect how the user presented in this session.
- Derive emotional state ONLY from explicit wording/tone in DIALOGUE_CHUNK.
- Keep it compact and conservative; do not over-infer.
- Use short strings for signals/patterns; avoid long narrative.

Rules:
- Add or modify goals only if the user clearly stated them.
- Add next actions only if the user explicitly committed to them.
- Mark actions done ONLY if user confirmed completion.
- Track blockers only if clearly described.
- Track preferences only if explicitly stated.
- Do NOT store secrets (API keys, passwords).
- Keep IDs stable if present.

Required keys (must always exist):
{
  "user_profile": { "name": null, "preferences": { "tone": "direct", "accountability": "high", "constraints": [] } },
  "goals": [],
  "current_focus": "",
  "next_actions": [],
  "plan": [],
  "blockers": [],
  "open_loops": [],
  "pattern_analysis": {
    "overall_tone": "neutral",
    "stress_level": 0,
    "dominant_emotions": [],
    "confidence_level": 0,
    "signals": [],
    "recurring_patterns": [],
    "last_session_notes": ""
  },
  "last_emotional_state": {
    "mood_label": "neutral",
    "valence": 0,
    "arousal": 0,
    "risk_flags": []
  },
  "last_session_summary": "",
  "updated_at": ""
}

Return JSON only.
"""

def update_coach_state(old_state, dialogue_chunk):
    messages = [
        {"role": "system", "content": MEMORY_UPDATER_PROMPT},
        {"role": "user", "content": f"OLD_COACH_STATE: {json.dumps(old_state)}\n\nDIALOGUE_CHUNK: {dialogue_chunk}"}
    ]

    # Using a model capable of good JSON generation
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        temperature=1,
        response_format={ "type": "json_object" }
    )
    
    new_state_json = response.choices[0].message.content
    try:
        return json.loads(new_state_json)
    except json.JSONDecodeError:
        print("Error decoding JSON from memory updater")
        return old_state

def load_user_state(user_id):
    if not user_id or user_id.strip() == "":
        return None, [], [], "Please enter a User ID to start."
    
    user_id = user_id.strip()
    state = get_or_create_coach_state(user_id)
    goals_preview = state.get('goals', [])[:3]
    goals_text = f" Goals: {goals_preview}" if goals_preview else ""
    return user_id, [], [], f"✓ Loaded state for user: {user_id}.{goals_text}"

def process_message(user_message, history, user_id, conv_history):
    if not user_id:
        return history, conv_history, "⚠ Please load a User ID first."
    if not user_message or user_message.strip() == "":
        return history, conv_history, ""
    
    coach_state = get_or_create_coach_state(user_id)
    messages = [{"role": "system", "content": COACH_SYSTEM_PROMPT}]
    messages.append({"role": "user", "content": f"COACH_STATE:\n{json.dumps(coach_state, indent=2)}"})
    messages.append({"role": "assistant", "content": "I've reviewed the COACH_STATE."})
    
    # clean_conv_history ensures we don't mix up types
    clean_conv_history = [m for m in conv_history if isinstance(m, dict) and 'role' in m] if conv_history else []
    for msg in clean_conv_history:
        messages.append(msg)
    messages.append({"role": "user", "content": user_message})
    
    try:
        response = get_message_completion(messages)
    except Exception as e:
        return history, conv_history, f"Error: {str(e)}"
    
    # Append to internal history (dicts for LLM)
    new_conv_history = clean_conv_history + [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": response}
    ]
    
    # For Gradio 6.x: Use messages format (dictionaries) for the chatbot display
    if history is None: history = []
    new_history = history + [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": response}
    ]
    
    return new_history, new_conv_history, ""

def update_memory(user_id, conv_history):
    if not user_id: return "⚠ No user loaded."
    if not conv_history: return "⚠ No conversation to save."
    
    recent_history = conv_history[-40:]
    dialogue_chunk = "\n".join([
        f"{msg['role'].capitalize()}: {msg['content']}" 
        for msg in recent_history if isinstance(msg, dict) and 'role' in msg
    ])
    
    old_state = get_or_create_coach_state(user_id)
    try:
        new_state = update_coach_state(old_state, dialogue_chunk)
        required_keys = ['user_profile', 'goals', 'pattern_analysis', 'last_emotional_state']
        if not isinstance(new_state, dict) or not all(k in new_state for k in required_keys):
            return "⚠ Memory update failed: invalid state."
        save_coach_state(user_id, new_state)
        return f"✓ Memory updated for {user_id}."
    except Exception as e:
        return f"⚠ Error: {str(e)}"

with gr.Blocks(title="AI Coach") as demo:
    gr.Markdown("# AI Coaching Assistant (v4-Gradio-6.x-Compatible)")
    gr.Markdown("Enter your User ID to load your coaching state.")
    
    current_user_id = gr.State(value=None)
    conversation_history = gr.State(value=[])
    
    with gr.Row():
        user_id_input = gr.Textbox(label="User ID", placeholder="Enter your name or ID...")
        load_btn = gr.Button("Load State", variant="primary")
    status_text = gr.Textbox(label="Status", interactive=False)
    
    # Gradio 6.x uses messages format (dict) by default
    chatbot = gr.Chatbot(label="Conversation", height=400)
        
    msg_input = gr.Textbox(label="Your message", placeholder="Type your message here...")
    
    with gr.Row():
        send_btn = gr.Button("Send", variant="primary")
        save_btn = gr.Button("💾 Update Memory", variant="secondary")
    gr.Markdown("*Click 'Update Memory' to save your session progress.*")
    
    load_btn.click(fn=load_user_state, inputs=[user_id_input], outputs=[current_user_id, chatbot, conversation_history, status_text])
    send_btn.click(fn=process_message, inputs=[msg_input, chatbot, current_user_id, conversation_history], outputs=[chatbot, conversation_history, msg_input])
    msg_input.submit(fn=process_message, inputs=[msg_input, chatbot, current_user_id, conversation_history], outputs=[chatbot, conversation_history, msg_input])
    save_btn.click(fn=update_memory, inputs=[current_user_id, conversation_history], outputs=[status_text])

demo.launch(share=False)


Gradio Version: 6.3.0
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
